# 📊 Data Processing for Phishing Detection

**Purpose**: Prepare the phishing detection dataset for model training.

This notebook handles:
- Loading the raw dataset from HuggingFace
- Preprocessing email content and labels
- Creating train/validation/test splits
- Analyzing dataset statistics
- Exporting to JSONL format
- Uploading processed data to S3

## Prerequisites
- SageMaker notebook instance or local environment
- AWS credentials configured
- Internet access to download HuggingFace datasets

## Expected Outputs
- `datasets/train.jsonl` - Training dataset (~23k samples)
- `datasets/validation.jsonl` - Validation dataset (~4k samples)
- `datasets/test.jsonl` - Test dataset (~3.7k samples)
- S3 paths stored in magic variables for next notebook

## Next Steps
After running this notebook, proceed to `02_model_training.ipynb`

---

## 1. Setup and Installation

Install only the dependencies needed for data processing.

In [ ]:
!pip uninstall -y autogluon-multimodal autogluon-core autogluon-features autogluon-tabular autogluon-timeseries
!pip install -Uq "datasets==4.3.0" "sagemaker==2.253.1" "pandas>=1.5.0" "scikit-learn>=1.3.0"

In [ ]:
import boto3
import sagemaker
import os
import json
import pandas as pd
import numpy as np
import random
from datasets import load_dataset, DatasetDict, Dataset, ClassLabel
from sagemaker.s3 import S3Uploader

## 2. SageMaker Configuration

Set up AWS session, role, and S3 bucket for data storage.

In [ ]:
region = boto3.Session().region_name
sess = sagemaker.Session(boto3.Session(region_name=region))
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

print(f"SageMaker role ARN: {role}")
print(f"SageMaker bucket: {sess.default_bucket()}")
print(f"SageMaker session region: {sess.boto_region_name}")

## 3. Dataset Loading

Load the [`drorrabin/phishing_emails-data`](https://huggingface.co/datasets/drorrabin/phishing_emails-data) dataset from HuggingFace.

### Dataset Overview:
- **Purpose**: Email phishing detection
- **Size**: ~27k training samples, ~3.7k test samples
- **Labels**: Binary (0=Safe, 1=Phishing)
- **Format**: Email text with classification labels
- **Source**: Optimized for LLM fine-tuning

In [ ]:
DATASET_NAME = "drorrabin/phishing_emails-data"
NUM_LABELS = 2  # Binary classification

LABEL_NAMES = {
    0: "Safe",      # Legitimate content
    1: "Phishing"   # Phishing/malicious content
}

print("Loading phishing dataset...")
print(f"Dataset: {DATASET_NAME}")

raw_dataset = load_dataset(
    path=DATASET_NAME,
    revision="9f364df1d4fa9ec50601bf2ddc293c71a469f749"
)

print(f"✅ Dataset loaded successfully!")
print(f"Available splits: {list(raw_dataset.keys())}")
print(f"Total training samples: {len(raw_dataset['train'])}")
print(f"Total test samples: {len(raw_dataset['test'])}")

In [ ]:
print("Sample data (raw):")
sample = random.choice(raw_dataset['train'])
print(json.dumps(sample, indent=2))

## 4. Data Preprocessing

The raw dataset format includes:
- **text**: Full prompt with email content + answer (e.g., "Is the following email safe or phishing??\n\n[EMAIL]\n\nEmail type is: [LABEL]")
- **email_type**: String label ("safe email" or "phishing email")

We need to:
1. Extract just the email content (remove prompt and answer)
2. Convert string labels to binary integers (0=safe, 1=phishing)

In [ ]:
def preprocess_phishing_email(example):
    """
    Extract email content and convert label to binary.
    
    Input format:
        'Is the following email safe or phishing??\n\n[EMAIL CONTENT]\n\nEmail type is: [LABEL]'
    
    Output:
        - text: Just the email content
        - label: 0 (safe) or 1 (phishing)
    """
    text = example['text']
    
    # Remove the prompt prefix
    if text.startswith("Is the following email safe or phishing??"):
        text = text.replace("Is the following email safe or phishing??", "").strip()
    
    # Remove the answer suffix
    if "Email type is:" in text:
        text = text.split("Email type is:")[0].strip()
    
    # Convert label to binary
    label = int('phishing' in example['email_type'].lower())
    
    return {
        'text': text,
        'label': label
    }

In [ ]:
print("Testing preprocessing...")
sample = random.choice(raw_dataset['train'])
processed = preprocess_phishing_email(sample)

print(f"\nOriginal length: {len(sample['text'])} chars")
print(f"Processed length: {len(processed['text'])} chars")
print(f"Original label: {sample['email_type']}")
print(f"Processed label: {processed['label']} ({LABEL_NAMES[processed['label']]})")
print(f"\nProcessed text preview (first 200 chars):\n{processed['text'][:200]}...")

In [ ]:
print("\nApplying preprocessing to entire dataset...")
raw_dataset = raw_dataset.map(
    preprocess_phishing_email,
    remove_columns=['email_type'],
    desc="Preprocessing"
)

print("Converting label to ClassLabel for stratified splitting...")
raw_dataset = raw_dataset.cast_column(
    'label',
    ClassLabel(names=['safe', 'phishing'])
)

print("✅ Preprocessing complete!")
print(f"\nProcessed sample:")
print(random.choice(raw_dataset['train']))

## 5. Dataset Analysis

Analyze the dataset to understand its characteristics.

In [ ]:
def analyze_label_distribution(dataset, split_name="train"):
    """
    Analyze and print label distribution for a dataset split.
    
    Args:
        dataset: HuggingFace DatasetDict, Dataset, or DataFrame with 'label' column
        split_name: Name of the split for display or to extract from DatasetDict
    
    Returns:
        dict: Statistics including counts and percentages
    """
    if isinstance(dataset, DatasetDict):
        df = pd.DataFrame(dataset[split_name])
    elif isinstance(dataset, Dataset):
        df = pd.DataFrame(dataset)
    else:
        df = dataset
    
    safe_count = (df['label'] == 0).sum()
    phishing_count = (df['label'] == 1).sum()
    safe_pct = (safe_count / len(df)) * 100
    phishing_pct = (phishing_count / len(df)) * 100
    
    print(f"\n📊 {split_name} Label Distribution:")
    print(f"  Total samples: {len(df)}")
    print(f"  Safe (0): {safe_count} ({safe_pct:.1f}%)")
    print(f"  Phishing (1): {phishing_count} ({phishing_pct:.1f}%)")
    
    return {
        'total': len(df),
        'safe_count': safe_count,
        'phishing_count': phishing_count,
        'safe_pct': safe_pct,
        'phishing_pct': phishing_pct
    }

In [ ]:
_ = analyze_label_distribution(raw_dataset, split_name="train")
_ = analyze_label_distribution(raw_dataset, split_name="test")

In [ ]:
def analyze_text_lengths(dataset, split_name="train"):
    """
    Analyze and print text length statistics with histogram.
    
    Args:
        dataset: HuggingFace DatasetDict, Dataset, or DataFrame with 'text' column
        split_name: Name of the split for display or to extract from DatasetDict
    
    Returns:
        dict: Statistics including mean, median, min, max, std
    """
    if isinstance(dataset, DatasetDict):
        df = pd.DataFrame(dataset[split_name])
    elif isinstance(dataset, Dataset):
        df = pd.DataFrame(dataset)
    else:
        df = dataset
    
    text_lengths = df['text'].str.len()
    
    print(f"\n📏 {split_name} Text Length Statistics:")
    print(f"  Mean: {text_lengths.mean():.0f} characters")
    print(f"  Median: {text_lengths.median():.0f} characters")
    print(f"  Min: {text_lengths.min()} characters")
    print(f"  Max: {text_lengths.max()} characters")
    print(f"  Std: {text_lengths.std():.0f} characters")
    
    print(f"\n  Character Length Distribution:")
    max_len = text_lengths.max()
    bins = [0, 200, 400, 600, 800, 1000, 1500, 2000]
    
    # Only add max_len if it's greater than the last bin
    if max_len > bins[-1]:
        bins.append(max_len)
    
    hist, bin_edges = np.histogram(text_lengths, bins=bins)
    
    for i in range(len(hist)):
        range_str = f"  {int(bin_edges[i])}-{int(bin_edges[i+1])}"
        bar = '█' * int(hist[i] / len(df) * 50)  # Scale to 50 chars max
        count = hist[i]
        pct = (count / len(df)) * 100
        print(f"{range_str:>12}: {bar} {count} ({pct:.1f}%)")
    
    return {
        'mean': text_lengths.mean(),
        'median': text_lengths.median(),
        'min': text_lengths.min(),
        'max': text_lengths.max(),
        'std': text_lengths.std()
    }

In [ ]:
_ = analyze_text_lengths(raw_dataset, split_name="train")
_ = analyze_text_lengths(raw_dataset, split_name="test")

## 6. Create Train/Validation/Test Splits

Split the training data to create a validation set for model evaluation during training.

In [ ]:
def prepare_phishing_dataset(raw_dataset, val_ratio=0.15, random_state=42):
    """
    Prepare train/val/test splits for phishing detection.
    
    Args:
        raw_dataset: HuggingFace DatasetDict with 'train' and 'test' splits,
                    with 'label' column as ClassLabel type
        val_ratio: Proportion of training data to use for validation
        random_state: Random seed for reproducibility
    
    Returns:
        DatasetDict with 'train', 'validation', and 'test' splits
    """
    print(f"\n📊 Original Dataset:")
    print(f"  Train: {len(raw_dataset['train'])} samples")
    print(f"  Test: {len(raw_dataset['test'])} samples")
    
    print(f"\n🔀 Creating stratified train/validation split ({int((1-val_ratio)*100)}/{int(val_ratio*100)})...")
    split_dataset = raw_dataset['train'].train_test_split(
        test_size=val_ratio,
        stratify_by_column='label',
        seed=random_state
    )
    
    train_dataset, validation_dataset, test_dataset = (
        split_dataset['train'],
        split_dataset['test'],
        raw_dataset['test']
    )
    
    print(f"\n📊 Final Dataset Splits:")
    print(f"  Train: {len(train_dataset)} samples")
    print(f"  Validation: {len(validation_dataset)} samples")
    print(f"  Test: {len(test_dataset)} samples")
    print(f"  Total: {len(train_dataset) + len(validation_dataset) + len(test_dataset)} samples")
    
    return DatasetDict({
        'train': train_dataset,
        'validation': validation_dataset,
        'test': test_dataset
    })

In [ ]:
print("\nPreparing dataset splits...")
dataset = prepare_phishing_dataset(raw_dataset)

## 7. Save Datasets Locally

Export datasets to JSONL format for training.

In [ ]:
os.makedirs('datasets', exist_ok=True)

dataset['train'].to_json('datasets/train.jsonl', lines=True)
dataset['validation'].to_json('datasets/validation.jsonl', lines=True)
dataset['test'].to_json('datasets/test.jsonl', lines=True)

print("✅ Datasets saved locally to ./datasets/")
print(f"  - train.jsonl: {len(dataset['train'])} samples")
print(f"  - validation.jsonl: {len(dataset['validation'])} samples")
print(f"  - test.jsonl: {len(dataset['test'])} samples")

## 8. Upload Datasets to S3

Upload processed datasets to S3 for SageMaker training job access.

In [ ]:
# Define S3 paths with consistent structure
s3_prefix = "phishing-detection"
data_s3_uri = f"s3://{sess.default_bucket()}/{s3_prefix}/data"

train_s3_uri = S3Uploader.upload(
    local_path='datasets/train.jsonl',
    desired_s3_uri=f"{data_s3_uri}/train"
)

val_s3_uri = S3Uploader.upload(
    local_path='datasets/validation.jsonl',
    desired_s3_uri=f"{data_s3_uri}/validation"
)

test_s3_uri = S3Uploader.upload(
    local_path='datasets/test.jsonl',
    desired_s3_uri=f"{data_s3_uri}/test"
)

print(f"📤 Datasets uploaded to S3:")
print(f"  Train: {train_s3_uri}")
print(f"  Validation: {val_s3_uri}")
print(f"  Test: {test_s3_uri}")

## 9. Store Variables for Next Notebook

Use IPython's `%store` magic to pass data to the training notebook.

In [ ]:
# Store S3 URIs
%store train_s3_uri
%store val_s3_uri
%store test_s3_uri

# Store model configuration
%store NUM_LABELS
%store LABEL_NAMES

# Store SageMaker configuration
%store region
%store role
%store sagemaker_session_bucket

print("\n✅ Variables stored successfully!")
print("\nStored variables:")
print(f"  - train_s3_uri: {train_s3_uri}")
print(f"  - val_s3_uri: {val_s3_uri}")
print(f"  - test_s3_uri: {test_s3_uri}")
print(f"  - NUM_LABELS: {NUM_LABELS}")
print(f"  - LABEL_NAMES: {LABEL_NAMES}")
print(f"  - region: {region}")
print(f"  - sagemaker_session_bucket: {sagemaker_session_bucket}")

## ✅ Data Processing Complete!

### What We Accomplished:
1. ✅ Loaded phishing email dataset from HuggingFace
2. ✅ Preprocessed email content and converted labels
3. ✅ Created stratified train/validation/test splits
4. ✅ Analyzed dataset statistics
5. ✅ Exported to JSONL format
6. ✅ Uploaded to S3
7. ✅ Stored variables for next notebook

### Next Steps:
**Proceed to `02_model_training.ipynb`** to fine-tune the Qwen2.5-1.5B model on this processed data.

---

**Estimated Time**: ~5-10 minutes  
**Estimated Cost**: ~$0.01 (S3 storage only)